<a href="https://colab.research.google.com/github/DocLock17/ISAR_Final/blob/main/20kRun4_MAT_Suite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modular Abstraction Transfer Suite

### A Deep Mind Based Experimental Platform For Reasearch In Abstaction And Generalization

We set out to assemble a team of Algorithms to fight the evil of an unseen game. Unfortunately, this task became nearly insurmountable. First iterations of the code were based implementations using a deque for replay memory, and using a single instance training loop. These systems did show promise in early experiments but were so inefficient that it would have likely taken months of training to reach a respectable level let alone expert level game play.

The solution it seems is multi-part, so we will spread out the explanations here.

|
Great References:

https://towardsdatascience.com/tutorial-double-deep-q-learning-with-dueling-network-architectures-4c1b3fb7f756

https://keras.io/examples/rl/deep_q_network_breakout/

https://stackoverflow.com/questions/15455048/releasing-memory-in-python

Official Citations in accompanying paper

#### Interactive Playgound (run after notebook)

In [ ]:
# # # Run From Top (Hint: It must be loaded commented first)
# myPlayground()

#### Selector Menu

In [ ]:
##   Game Selector
ENVIRONMENT_NAME = 'BreakoutNoFrameskip-v4' #@param ['Atlantis-v0', 'DemonAttack-v0', 'Phoenix-v0', 'Riverraid-v0', 'Solaris-v0', 'Asterix-v0', 'Breakout-v0', 'Boxing-v0', 'Pong-v0', 'BattleZone-v0', 'SpaceInvaders-v0', 'BeamRider-v0','AtlantisNoFrameskip-v4', 'DemonAttackNoFrameskip-v4', 'PhoenixNoFrameskip-v4', 'RiverraidNoFrameskip-v4', 'SolarisNoFrameskip-v4', 'AsterixNoFrameskip-v4', 'BreakoutNoFrameskip-v4', 'BoxingNoFrameskip-v4', 'PongNoFrameskip-v4', 'BattleZoneNoFrameskip-v4', 'SpaceInvadersNoFrameskip-v4', 'BeamRiderNoFrameskip-v4']

# Render gameplay in cell or viewer
RENDER = False #@param {type:"boolean"}

# Slow down game play and discontinue training for observation
OBSERVATION_MODE = False #@param {type:"boolean"}

# Display Text Output to Observe Activations and Q Updates
DIAGNOSTIC_MODE = False #@param {type:"boolean"}

## Chippie the Progress Bot Settings       (Name courtesy of: Kylie Locker)
CHIPPIE_PROGESS_REPORTS = True #@param {type:"boolean"}
# Row for progress bot stacking ( 1, 8, 12 )
MAX_CHIPPIES_ROW = 8 #@param {type:"integer"}

# Provide GPU information
USE_GPU_SUPPORT = True #@param {type:"boolean"}


## Data Settings
# Import and Unzip Hosted Dataset
CREATE_MISSING_DIRECTORIES = True #@param {type:"boolean"}
IMPORT_MAIN_DATA = True #@param {type:"boolean"}
IMPORT_SENTIMENT_DATA = True #@param {type:"boolean"}
IMPORT_MODEL_DATA = True #@param {type:"boolean"}


## Agent Settings
# Parameters
n_episodes = 20000 #@param {type:"integer"}
BATCH_SIZE =  32#@param {type:"integer"}
action_steps =  4#@param {type:"integer"}
skip_start = 1 #@param {type:"integer"}
agent_gamma = 0.985 #@param {type:"slider", min:0, max:1, step:0.0001}
agent_epsilon = 1.0 #@param {type:"slider", min:0, max:1, step:0.0001}
agent_epsilon_decay = 0.9999 #@param {type:"slider", min:0.5, max:1.000, step:0.0001}
agent_epsilon_min = 0.1 #@param {type:"slider", min:0, max:1.000, step:0.0001}
agent_learning_rate = 0.00025 #@param {type:"number", min:0.0000, max:0.0200, step:0.00001}
# lr=0.00042 # lr=0.00125 #lr=0.00025
# agent_learning_rate = 0.00025 # works??
## Buffer Settings
STARTING_MEMORY_SIZE = 75000#@param {type:"integer"}
MAX_MEMORY_LENGTH = 100000 #@param {type:"integer"}
Q_UPDATE_FREQUENCY = 10000 #@param {type:"integer"}

# Pre Buffer Guidance
WINDOW_SIZE = (84, 84)

# Sub-Model Settings
SLICE_ONE_NAME = "s1_MAT_ImageClassifier_v5" #@param {type:"string"}
SLICE_ONE_TRAINABLE = False #@param {type:"boolean"}
SLICE_ONE_CHECKPOINT = False #@param {type:"boolean"}

SLICE_TWO_NAME = "s2_MAT_ImageClassifier_v5" #@param {type:"string"}
SLICE_TWO_TRAINABLE = False #@param {type:"boolean"}
SLICE_TWO_CHECKPOINT = False #@param {type:"boolean"}

SLICE_THREE_NAME = "s3_MAT_ImageClassifier_v2" #@param {type:"string"}
SLICE_THREE_TRAINABLE = True #@param {type:"boolean"}
SLICE_THREE_CHECKPOINT = True #@param {type:"boolean"}

FULL_MODEL_NAME = "s2_MAT_Agent_Testing_v2" #@param {type:"string"}
FULL_MODEL_TRAINABLE = True #@param {type:"boolean"}
FULL_MODEL_CHECKPOINT = True #@param {type:"boolean"}

CHECKPOINT_FREQUENCY =  1000#@param {type:"integer"}

SAVE_AGENT = True #@param {type:"boolean"}

## Game selector feedback
print("Selected Game: " + ENVIRONMENT_NAME)


Selected Game: BreakoutNoFrameskip-v4


#### Setup Model Output Directory

In [ ]:
%%capture

import os

try:
  import google.colab
  !pip install baselines
  COLAB = True
  
except:
  !pip install gdown
  COLAB = False

model_dir = 'ISAR_Model_Data'
data_dir = 'ISAR_Main_Classification'
transfer_dir = 'ISAR_Sentiment_Transfer'


if IMPORT_MAIN_DATA :
    if not os.path.exists(data_dir): ## Make it if it doesn't exist
      !gdown https://drive.google.com/uc?id=1P7o1x4ZpPbd16VQDwaMzllbN-tlfqqIH
      !unzip ISAR_Main_Classification.zip
      !rm ISAR_Main_Classification.zip
if IMPORT_SENTIMENT_DATA:
    if not os.path.exists(transfer_dir): ## Make it if it doesn't exist
      !gdown https://drive.google.com/uc?id=1UDUNnw04q5cvms5ibM6pNn-wXtJphXxZ
      !unzip ISAR_Sentiment_Transfer.zip
      !rm ISAR_Sentiment_Transfer.zip
if IMPORT_MODEL_DATA:
    if not os.path.exists(model_dir): ## Make it if it doesn't exist
      !gdown https://drive.google.com/uc?id=1DIc_J6XyKzNDSMjSbGkt3vzQep4YUEYU
      !unzip ISAR_Model_Data.zip
      !rm ISAR_Model_Data.zip
      

In [ ]:
if CREATE_MISSING_DIRECTORIES:  
  if not os.path.exists(data_dir): ## Make it if it doesn't exist
    print("Creating Main Data Directory")
    os.makedirs(data_dir)
  else:
    print("Main Data Directory Found")
  if not os.path.exists(transfer_dir): ## Make it if it doesn't exist
    print("Creating Transfer Data Directory")
    os.makedirs(transfer_dir)
  else:
    print("Transfer Data Directory Found")
  if not os.path.exists(model_dir): ## Make it if it doesn't exist
    print("Creating Model Directory")
    os.makedirs(model_dir)
  else:
    print("Model Directory Found")
    

Main Data Directory Found
Transfer Data Directory Found
Model Directory Found


#### Setup Imports

In [ ]:
## Begin by importing . . .  oh . . . everything!
try:
  import math
  import random
  import numpy as np

  import glob
  import io
  import base64
  from time import sleep

  from collections import deque

  import gym
  import tensorflow as tf
  import tensorflow_hub as hub
  from tensorflow.keras.models import Sequential
  from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
  from tensorflow.keras.optimizers import Nadam, Adam
  from tensorflow import keras

  import matplotlib
  import matplotlib.pyplot as plt
  %matplotlib inline

  from IPython.display import HTML
  from IPython import display as ipythondisplay
  from IPython.display import clear_output
  
  from gym.wrappers.atari_preprocessing import AtariPreprocessing
  from baselines.common.atari_wrappers import make_atari, EpisodicLifeEnv, WarpFrame, ScaledFloatFrame, ClipRewardEnv, FrameStack, FireResetEnv



except:
  # %%capture
  if COLAB:
    ## For colab we must install some dependancies
    # !pip install baselines
    !apt-get install -y xvfb x11-utils
    ## Next we will need to install a virtual display and correct Open AI installation
    !pip install gym[all]==0.17.3
    !pip install pyvirtualdisplay==0.2.* 
    !pip install PyOpenGL==3.1.* 
    !pip install PyOpenGL-accelerate==3.1.*
    !pip install pyglet
    # So let's setup the virtual display
    import pyvirtualdisplay
    # use False with Xvfb
    _display = pyvirtualdisplay.Display(visible=False, size=(1400, 900))
    _ = _display.start()
    # Now Check the Display
    !echo $DISPLAY

if USE_GPU_SUPPORT:
  print("TF version:", tf.__version__)
  print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

  # Set Memory Growth
  gpus = tf.config.experimental.list_physical_devices('GPU')
  if gpus:
    try:
      # Currently, memory growth needs to be the same across GPUs
      for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
        print(gpu)
      logical_gpus = tf.config.experimental.list_logical_devices('GPU')
      print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
      # Memory growth must be set before GPUs have been initialized
      print(e)

TF version: 2.4.1
Num GPUs Available:  2
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')
2 Physical GPUs, 2 Logical GPUs


#### The Wrappers
So here we will use some wrappers from the baselines package. This broke my Tensorflow installation on my home server so I avoided wrappers during the majority of this project however, in the end using the optimized implementations in the wrapper drove speed up 1000% or so. So, if you are implementing your own version of this experiment, I cannot stress enough how important it is to study up on these wrappers to start.

In [ ]:
# Make Environment

env = make_atari(ENVIRONMENT_NAME)
env = EpisodicLifeEnv(env)

if 'FIRE' in env.unwrapped.get_action_meanings():
        env = FireResetEnv(env)

env = WarpFrame(env)

env = ScaledFloatFrame(env)

env = ClipRewardEnv(env)

env = FrameStack(env, 3)

# myWrappedEnv = FrameStack(atariWrap, 3)                  env_wrappers=[lambda env: ActionRepeat(env, times=4)])
env.seed(42)
np.random.seed(42)
tf.random.set_seed(42)
# env = gym.make(ENVIRONMENT_NAME)
state_size = (84, 84, 3)
action_size = env.action_space.n


#### Setup Progress Bot

Here we find Chippie the progress bot, I in the hours of waiting for code to test and the constant need to debug I created a simple progress bar that slowly turned into an entire diagnostic companion. However, as debugging reveiled greater and greater speed enhancements, Chippies output is limited to every 250 Episodes now and his diagnostic features removed or turned off.

As one benefit, note that when using Gradient tape we don't get standard output from tensorflow so Chippie's periodic reports include leading and trailing averages of the last 1000, and previous 1000 loss calculations allowing us to see how our agent is progressing through the training process. If you are tinkering with the code you might try out the diagnostic mode to get a snap shot of the DQN learning process as it is actually happening.

In [ ]:
# import gc
import sys

class ChippieProgressBot:

    def __init__(self, window_size=100, row_configuration=8, log_size=2000):
        self.rowCount = 0
        self.miniScore = 0
        self.movingAverage = []
        self.windowSize = window_size
        self.facelist1 = ["~{0_0}~","~{o_o}~","~{o_0}~","~{0_o}~"]
        self.row_configuration = row_configuration
        self.lossLog = []
        self.logSize = log_size
        self.lessChippies = True
        self.skippedChippies = 250

    def training(self, score):
      if not self.lessChippies:
        if self.miniScore < score:
            self.miniScore += 1
            print(self.facelist1[int(self.miniScore % 4)] + str(int(score))+"   ", end='')
            self.rowCount += 1
            if self.rowCount >= self.row_configuration:
                print("\n")
                self.rowCount = 0
        else:
           return
      else:
        return


    def q_update(self):
      if not self.lessChippies:
        print("""  {-_-}    """, end='')
        self.rowCount += 1
        if self.rowCount >= MAX_CHIPPIES_ROW:
            print("\n")
            self.rowCount = 0
      else:
        return

    def dead(self, score, totalScore, episode, completion_target,  survived, experiance, memory, epsilon):
      
        if len(self.movingAverage) >= self.windowSize:
          del self.movingAverage[:1]
        self.movingAverage.append(score)
        if len(self.lossLog) > 2:
          trailing = sum(self.lossLog[:len(self.lossLog)//2])/(len(self.lossLog)*0.5)

          leading = sum(self.lossLog[len(self.lossLog)//2:])/(len(self.lossLog)*0.5)

        self.rowCount = 0
        self.miniScore = 0
        if not self.lessChippies:

          # running_reward = np.mean(episode_reward_history)
          print(""" `{x_X}~   """+"\n")

          print("Episode: {}/{}, Episode Score: {}, Avg Episode Score: {:.4}, Survival Time: {}"
          .format(episode+1, completion_target, score, totalScore/(episode+1), survived)+"\n")

          print("Your agent has a running average Score per 100 episodes of: \{^,^}~"+"{:.3}"
          .format(sum(self.movingAverage)/len(self.movingAverage))+"\n")

          print("Total Steps: {}, Memory Size: {}, Current Epsilon Value: {:.2}, Leading Loss: {:.4}/ Trailing Loss: {:.4}"
          .format(experiance, memory, epsilon, leading, trailing)+"\n\n")

        
        else:
          if (episode+1) % self.skippedChippies == 0:

            print("\n"+"Episode: {}/{}, Episode Score: {}, Avg Episode Score: {:.4}, Survival Time: {}"
            .format(episode+1, completion_target, score, totalScore/(episode+1), survived))

            print("Your agent has a running average Score per 100 episodes of: \{^,^}~"+"{:.3}"
            .format(sum(self.movingAverage)/len(self.movingAverage)))

            print("Total Steps: {}, Memory Size: {}, Current Epsilon Value: {:.2}, Leading Loss: {:.4}/ Trailing Loss: {:.4}"
            .format(experiance, memory, epsilon, leading, trailing)) 
          else:
            return


    def logLoss(self, loss):
        if len(self.lossLog) >= self.logSize:
            del self.lossLog[:1]
        self.lossLog.append(loss)
        # trailing = sum(self.lossLog[:len(self.lossLog)//2])/(len(self.lossLog)*0.5)
        # print(leading)
        # leading = sum(self.lossLog[len(self.lossLog)//2:])/(len(self.lossLog)*0.5)
        # print(trailing)

# chippie.training(score)
chippie = ChippieProgressBot(window_size=100, row_configuration=MAX_CHIPPIES_ROW)

#### The Frame PreProcessor and Ring Buffer that was: New Replay Memory

After managing to make batches acceptable to the TensorFlow's .fit() method, we noticed a massive increase in training speed. The victory was short lived, however, as memory usage skyrocket outside the limits of both colaboratory and one local workstation (128GB). So, both the deque and image pre-processors had to be re-examined. This resulted in a home brewed ring buffer and numpy base preprocessor being implemented, which monumentally increased speed. Implementing the image pre-processor buffer using numpy arrays saved a few extra operations so I am confident this was all we cound do. However, in the final edit we realized that the FrameStack Wrapper from the Baselines package could be instantiated with 3 frame setting rendering this preprocessor no longer necessary and implemented a rotating list ring buffer based on Geron's implementation which provides the random access benefits of our ring buffer that doesn' waste cpu cycles moving indices.

In [ ]:
# Geron provides a great note on this buffer design that creates significant speed increases.
class ReplayMemory:
    def __init__(self, max_size):
        self.buffer = np.empty(max_size, dtype=np.object)
        self.max_size = max_size
        self.index = 0
        self.size = 0

    def append(self, obj):
        self.buffer[self.index] = obj
        self.size = min(self.size + 1, self.max_size)
        self.index = (self.index + 1) % self.max_size

    def sample(self, batch_size):
        indices = np.random.randint(self.size, size=batch_size)
        return self.buffer[indices]

#### Define the agent

In the search for optimization, we found a few pointers to use variance scaling in layer declaration. I include the links for a deeper dive, however essentially this means that the initiallization of weights and biases are resricted to a range with a standard deviation of <= 2.0. This makes sense since the Q-values we are hoping for will yield need to be close to each other to promote further explorations.


After, careful examination of a much faster implementation attempting to solve the traditional 4-frame 84x84 version was very helpful. So, we took time to implement the network using GradientTape(), which made the code run about twice as fast.

In [ ]:
class DQNAgent:

    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.replay_memory = ReplayMemory(max_size=100000)

        
        self.update_rate = Q_UPDATE_FREQUENCY # Number of steps until updating the target network

        self.gamma = agent_gamma # decay or discount rate

        self.epsilon = agent_epsilon # exploration rate
        self.epsilon_decay = agent_epsilon_decay # exploration decay
        self.epsilon_min = agent_epsilon_min # min exploration

        self.learning_rate = agent_learning_rate # SGD or Nadam rate param

        # self.model, self.sliceOne, self.sliceTwo, self.sliceThree = self._build_new_model()
        # self.target_model, self.target_sliceOne, self.target_sliceTwo, self.target_sliceThree = self._build_new_model()

        # self.model, self.sliceOne, self.sliceTwo, self.sliceThree = self._build_1S_transfer_model()
        # self.target_model, self.target_sliceOne, self.target_sliceTwo, self.target_sliceThree = self._build_1S_transfer_model()

        self.model, self.sliceOne, self.sliceTwo, self.sliceThree = self._build_2S_transfer_model()
        self.target_model, self.target_sliceOne, self.target_sliceTwo, self.target_sliceThree = self._build_2S_transfer_model()

        # self.model, self.sliceOne, self.sliceTwo, self.sliceThree = self._build_3S_transfer_model()
        # self.target_model, self.target_sliceOne, self.target_sliceTwo, self.target_sliceThree = self._build_3S_transfer_model()

        self.target_model.set_weights(self.model.get_weights()) # create Q-target network
        
        self.model.summary()

  
    def _build_new_model(self): # private method
        # Slice One
        s1_input_layer = tf.keras.Input(shape=WINDOW_SIZE + (3,))
        s1_conv1 = tf.keras.layers.Conv2D(32, kernel_size=(8, 8),strides=4, activation='relu', name='S1_Conv1', \
                                    kernel_initializer=tf.keras.initializers.VarianceScaling(scale=2.0))(s1_input_layer)
        s1_output = tf.keras.layers.Conv2D(64, kernel_size=(4, 4),strides=2, activation='relu', name='S1_Conv2', \
                                    kernel_initializer=tf.keras.initializers.VarianceScaling(scale=2.0))(s1_conv1)
        s1_model = tf.keras.Model(inputs=s1_input_layer, outputs=s1_output)
        # s1_model.summary()

       # Slice Two
        s2_input_layer = tf.keras.Input(shape=(9,9,64))
        s2_conv1 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3),strides=1, activation='relu', \
                                    kernel_initializer=tf.keras.initializers.VarianceScaling(scale=2.0))(s2_input_layer)
        s2_flat_1 = tf.keras.layers.Flatten(name='S2_Flat1')(s2_conv1)
        s2_output = tf.keras.layers.Dense(512, activation='relu', name='s2_Dense_1', \
                                    kernel_initializer=tf.keras.initializers.VarianceScaling(scale=2.0))(s2_flat_1)
        s2_model = tf.keras.Model(inputs=s2_input_layer, outputs=s2_output)
        s2_model._name = SLICE_TWO_NAME
        # s2_model.summary()

        # Slice Three
        s3_input_layer = tf.keras.Input(shape=(512))
        s3_output = tf.keras.layers.Dense(self.action_size, activation='linear', \
                                    kernel_initializer=tf.keras.initializers.VarianceScaling(scale=2.0))(s3_input_layer)
        s3_model = tf.keras.Model(inputs=s3_input_layer, outputs=s3_output)
        s3_model._name = SLICE_THREE_NAME
        # s3_model.summary()

        full_model_input = tf.keras.Input(shape=(WINDOW_SIZE+(3,)))
        s1_pass = s1_model(full_model_input) #, training=False)
        s2_pass = s2_model(s1_pass)
        s3_final_output = s3_model(s2_pass)

        
        full_model = tf.keras.Model(inputs=full_model_input, outputs=s3_final_output)
        full_model._name = FULL_MODEL_NAME
        # full_model.summary()

        s1_model.trainable = SLICE_ONE_TRAINABLE # ref param above
        s2_model.trainable = SLICE_TWO_TRAINABLE # ref param above
        s3_model.trainable = SLICE_THREE_TRAINABLE # ref param above
        full_model.trainable = FULL_MODEL_TRAINABLE # ref param above
        
        # # Using Nadam because its awesome
        self.optimizer = keras.optimizers.Nadam(learning_rate=self.learning_rate, clipnorm=1.0)
        # # Using Huber loss for stability
        self.loss_function = keras.losses.Huber(reduction=tf.keras.losses.Reduction.SUM)
        
        return full_model, s1_model, s2_model, s3_model
        

    def _build_1S_transfer_model(self): # private method    

        # Slice One
        s1_input_layer = tf.keras.Input(shape=WINDOW_SIZE + (3,))
        s1_model = tf.keras.models.load_model(model_dir + '/' + SLICE_ONE_NAME)
        s1_model._name = SLICE_ONE_NAME
        # s1_model.summary()

        # Slice Two
        s2_input_layer = tf.keras.Input(shape=(9,9,64))
        s2_conv1 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3),strides=1, activation='relu', \
                                    kernel_initializer=tf.keras.initializers.VarianceScaling(scale=2.0))(s2_input_layer)
        s2_flat_1 = tf.keras.layers.Flatten(name='S2_Flat1')(s2_conv1)
        s2_output = tf.keras.layers.Dense(512, activation='relu', name='s2_Dense_1', \
                                    kernel_initializer=tf.keras.initializers.VarianceScaling(scale=2.0))(s2_flat_1)
        s2_model = tf.keras.Model(inputs=s2_input_layer, outputs=s2_output)
        s2_model._name = SLICE_TWO_NAME
        # s2_model.summary()

        # Slice Three
        s3_input_layer = tf.keras.Input(shape=(512))
        s3_output = tf.keras.layers.Dense(self.action_size, activation='linear', \
                                    kernel_initializer=tf.keras.initializers.VarianceScaling(scale=2.0))(s3_input_layer)
        s3_model = tf.keras.Model(inputs=s3_input_layer, outputs=s3_output)
        s3_model._name = SLICE_THREE_NAME
        # s3_model.summary()

        full_model_input = tf.keras.Input(shape=(WINDOW_SIZE+(3,)))
        s1_pass = s1_model(full_model_input) #, training=False)
        s2_pass = s2_model(s1_pass)
        s3_final_output = s3_model(s2_pass)

        
        full_model = tf.keras.Model(inputs=full_model_input, outputs=s3_final_output)
        full_model._name = FULL_MODEL_NAME
        # full_model.summary()

        s1_model.trainable = SLICE_ONE_TRAINABLE # ref param above
        s2_model.trainable = SLICE_TWO_TRAINABLE # ref param above
        s3_model.trainable = SLICE_THREE_TRAINABLE # ref param above
        full_model.trainable = FULL_MODEL_TRAINABLE # ref param above
        
        # # Using Nadam because its awesome
        self.optimizer = keras.optimizers.Nadam(learning_rate=self.learning_rate, clipnorm=1.0)
        # # Using Huber loss for stability
        self.loss_function = keras.losses.Huber(reduction=tf.keras.losses.Reduction.SUM)
        
        return full_model, s1_model, s2_model, s3_model


    def _build_2S_transfer_model(self): # private method    

        # Slice One
        s1_input_layer = tf.keras.Input(shape=WINDOW_SIZE+(3,))
        s1_model = tf.keras.models.load_model(model_dir + '/' + SLICE_ONE_NAME)
        s1_model._name = SLICE_ONE_NAME
        # s1_model.summary()

        # Slice Two
        s2_input_layer = tf.keras.Input(shape=(9,9,64))
        # s2_input_layer = tf.keras.Input(shape=(s1_model.shape[1::1]))
        s2_model = tf.keras.models.load_model(model_dir + '/' + SLICE_TWO_NAME)
        s2_model._name = SLICE_TWO_NAME
        # s2_model.summary()

        # Slice Three
        s3_input_layer = tf.keras.Input(shape=(512))
        s3_output = tf.keras.layers.Dense(self.action_size, activation='linear', \
                                    kernel_initializer=tf.keras.initializers.VarianceScaling(scale=2.0))(s3_input_layer)
        s3_model = tf.keras.Model(inputs=s3_input_layer, outputs=s3_output)
        s3_model._name = SLICE_THREE_NAME
        # s3_model.summary()

        full_model_input = tf.keras.Input(shape=(WINDOW_SIZE+(3,)))
        s1_pass = s1_model(full_model_input) #, training=False)
        s2_pass = s2_model(s1_pass)
        s3_final_output = s3_model(s2_pass)

        
        full_model = tf.keras.Model(inputs=full_model_input, outputs=s3_final_output)
        full_model._name = FULL_MODEL_NAME
        # full_model.summary()

        s1_model.trainable = SLICE_ONE_TRAINABLE # ref param above
        s2_model.trainable = SLICE_TWO_TRAINABLE # ref param above
        s3_model.trainable = SLICE_THREE_TRAINABLE # ref param above
        full_model.trainable = FULL_MODEL_TRAINABLE # ref param above
        
        # # Using Nadam because its awesome
        self.optimizer = keras.optimizers.Nadam(learning_rate=self.learning_rate, clipnorm=1.0)
        # # Using Huber loss for stability
        self.loss_function = keras.losses.Huber(reduction=tf.keras.losses.Reduction.SUM)
        
        return full_model, s1_model, s2_model, s3_model


    def _build_3S_transfer_model(self): # private method    

        # Slice One
        s1_input_layer = tf.keras.Input(shape=WINDOW_SIZE + (3,))
        s1_model = tf.keras.models.load_model(model_dir + '/' + SLICE_ONE_NAME)
        s1_model._name = SLICE_ONE_NAME
        # s1_model.summary()

        # Slice Two
        s2_input_layer = tf.keras.Input(shape=(9,9,64))
        s2_model = tf.keras.models.load_model(model_dir + '/' + SLICE_TWO_NAME)
        s2_model._name = SLICE_TWO_NAME
        # s2_model.summary()

        # Slice Three                                   #### ndim 4?
        s3_input_layer = tf.keras.Input(shape=(512))
        s3_model = tf.keras.models.load_model(model_dir + '/' + SLICE_THREE_NAME)
        s3_model._name = SLICE_THREE_NAME
        # s3_model.summary()

        full_model_input = tf.keras.Input(shape=(WINDOW_SIZE+(3,)))
        s1_pass = s1_model(full_model_input) #, training=False)
        s2_pass = s2_model(s1_pass)
        s3_final_output = s3_model(s2_pass)

        
        full_model = tf.keras.Model(inputs=full_model_input, outputs=s3_final_output)
        full_model._name = FULL_MODEL_NAME
        # full_model.summary()

        s1_model.trainable = SLICE_ONE_TRAINABLE # ref param above
        s2_model.trainable = SLICE_TWO_TRAINABLE # ref param above
        s3_model.trainable = SLICE_THREE_TRAINABLE # ref param above
        full_model.trainable = FULL_MODEL_TRAINABLE # ref param above
        
        # # Using Nadam because its awesome
        self.optimizer = keras.optimizers.Nadam(learning_rate=self.learning_rate, clipnorm=1.0)
        # # Using Huber loss for stability
        self.loss_function = keras.losses.Huber(reduction=tf.keras.losses.Reduction.SUM)
        
        return full_model, s1_model, s2_model, s3_model

    def remember(self, state, action, reward, next_state, done):
        # My newly reimplemented memory, Hopefully it makes us faster
        self.replay_memory.append((state, action, reward, next_state, done))

    def jit_sampler(self, batch_size):
        batch = self.replay_memory.sample(batch_size)
        state_batch, action_batch, reward_batch, next_state_batch, done_batch = [
                np.array([exp[fidx] for exp in batch])
                for fidx in range(5)]
        return state_batch, action_batch, reward_batch, next_state_batch, done_batch


    def train(self, batch_size): # method that trains agent model
        ### The New Implementation
        state_batch, action_batch, reward_batch, next_state_batch, done_batch = self.jit_sampler(batch_size)

        ## This is where all the magic happens
        qValueF = self.target_model.predict_on_batch(next_state_batch) # approximate future reward

        qValueUpdate = reward_batch + self.gamma * tf.reduce_max(qValueF, axis=1)

        qValueUpdate = qValueUpdate * (1 - done_batch) - done_batch

        oneHotMask = tf.one_hot(action_batch, self.action_size)

        with tf.GradientTape() as tape:
            # Train the model on the states and updated Q-values
            OqValue = self.model(state_batch)

            # Apply the masks to the Q-values to get the Q-value for action taken
            OqValue_action = tf.reduce_sum(tf.multiply(OqValue, oneHotMask), axis=1)

            # Calculate loss between new Q-value and old Q-value
            loss = self.loss_function(qValueUpdate, OqValue_action)
            if CHIPPIE_PROGESS_REPORTS:
                  chippie.logLoss(loss)

        if DIAGNOSTIC_MODE:
          print("Q-Values: ", qValueF[1])
          print("Q-Values Update: ", qValueUpdate[1])
          print("Masked Update:   ", oneHotMask[1])
          print("Old Activations: ", OqValue[1])

        # Backpropagation
        grads = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))

        if DIAGNOSTIC_MODE:
          newQ = self.model.predict_step(state_batch)
          print("New Activations: ", newQ[1], "\n")

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


    def act(self, state):
        if np.random.rand() <= self.epsilon: 
            return random.randrange(self.action_size) # Do something stupid! (i.e. take a random action)
        action_values = self.model.predict_step(tf.expand_dims(tf.convert_to_tensor(state), 0))
        return tf.argmax(action_values[0]).numpy()


    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())
    

    def save(self, name):
        self.model.save_weights(name)


    def load(self, name):
        self.model.load_weights(name)

## Intialize Our Agent
agent = DQNAgent(state_size, action_size)


Model: "s2_MAT_Agent_Testing_v2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 84, 84, 3)]       0         
_________________________________________________________________
s1_MAT_ImageClassifier_v5 (F (None, 9, 9, 64)          39008     
_________________________________________________________________
s2_MAT_ImageClassifier_v5 (F (None, 512)               1643072   
_________________________________________________________________
s3_MAT_ImageClassifier_v2 (F (None, 4)                 2052      
Total params: 1,684,132
Trainable params: 1,684,132
Non-trainable params: 0
_________________________________________________________________


#### Define Training Loop Environment

Getting your reward function is pretty paramount. In this case we will clip rewards to +1 and -1, with -1 being awarded for reaching a terminal state.

In [ ]:
def myPlayground():

    # Empty Rewards Counter
    reward = 0
    all_rewards = 0
    total_time = 0
    done = False

    # Display in Colab
    if COLAB:
        if RENDER == True:
            done, ax = plt.subplots(1, 1)
            img = ax.imshow(env.render('rgb_array'))    

    # Make Some Starting data
    state = env.reset() # Reset state for new episode
    while agent.replay_memory.size < STARTING_MEMORY_SIZE:
      action = random.randrange(agent.action_size)
      next_frame, reward, done, _ = env.step(action) # Agent sends action to env wrapper and gets feedback

      next_state = next_frame
      if reward > 0.0: # Check and modify reward
        reward = 1.0
      if reward < 0.0:
        reward = -1.0
      if done:
        reward = -1.0
      agent.remember(state, action, reward, next_state, done) # Store sequence in replay memory
      state = next_state # Update state
      if done:
        state = env.reset()
    print("Starting Frames Acquired")

    # Training Loop
    for e in range(n_episodes): ## Go Eat Cherries
        done = False
        time = 0
        game_score = 0
        reward = 0


        state = env.reset() # Reset state for new episode
        
        for skip in range(skip_start): # Skip the start of each game
            env.step(0)

        while not done:
        
            time += 1
            total_time += 1

            # Display
            if RENDER:
                if COLAB:
                    img.set_data(env.render(mode='rgb_array')) 
                    ax.axis('off')
                    ipythondisplay.display(plt.gcf())
                    ipythondisplay.clear_output(wait=True)
                else:
                    env.render()
                    if OBSERVATION_MODE:
                        sleep(0.02)

            # Update Target Network
            if total_time % agent.update_rate == 0:
                agent.update_target_model()
                if CHIPPIE_PROGESS_REPORTS:
                  chippie.q_update()

            # Transition Dynamics
            action = agent.act(state) # Get action from agent
            next_frame, reward, done, _ = env.step(action) # Agent sends action to env wrapper and gets feedback
            next_state = next_frame
            
            # Sternly Validate Reward
            if reward > 0.0: 
              reward = 1.0
              game_score += reward
            if reward < 0.0:
              reward = -1.0
            if done:
              reward = -1.0
              all_rewards += game_score
              if CHIPPIE_PROGESS_REPORTS:
                chippie.dead(score=game_score, totalScore=all_rewards, episode=e, completion_target=n_episodes, survived=time, 
                              experiance=total_time, memory=agent.replay_memory.size, epsilon=agent.epsilon)
              agent.remember(state, action, reward, next_state, done) # Store death in replay memory
              break

            agent.remember(state, action, reward, next_state, done) # Store sequence in replay memory
            state = next_state # Update state

            if agent.replay_memory.size > STARTING_MEMORY_SIZE:
              if not OBSERVATION_MODE:
                if total_time % action_steps == 0:
                  agent.train(BATCH_SIZE)
                  if CHIPPIE_PROGESS_REPORTS:
                    chippie.training(game_score)

        if e % CHECKPOINT_FREQUENCY == 0 and agent.replay_memory.size > STARTING_MEMORY_SIZE+100:
          print("")
          if SLICE_ONE_CHECKPOINT:
              saved_model_path = model_dir + "/ISAR_Production_Models/" + ENVIRONMENT_NAME + SLICE_ONE_NAME + '{:04d}'.format(int(e/CHECKPOINT_FREQUENCY)) 
              tf.saved_model.save(agent.sliceOne, saved_model_path)

          if SLICE_TWO_CHECKPOINT:
              saved_model_path = model_dir + "/ISAR_Production_Models/" + ENVIRONMENT_NAME + SLICE_TWO_NAME + '{:04d}'.format(int(e/CHECKPOINT_FREQUENCY))
              tf.saved_model.save(agent.sliceTwo, saved_model_path)
          
          if SLICE_THREE_CHECKPOINT:
              saved_model_path = model_dir + "/ISAR_Production_Models/" + ENVIRONMENT_NAME + SLICE_TWO_NAME + '{:04d}'.format(int(e/CHECKPOINT_FREQUENCY))
              tf.saved_model.save(agent.sliceTwo, saved_model_path)

          if FULL_MODEL_CHECKPOINT:
              saved_model_path = model_dir + "/ISAR_Production_Models/" + ENVIRONMENT_NAME + FULL_MODEL_NAME + '{:04d}'.format(int(e/CHECKPOINT_FREQUENCY))
              tf.saved_model.save(agent.model, saved_model_path)

          if SAVE_AGENT:
              agent.save(model_dir+ "/ISAR_Production_Models/" + ENVIRONMENT_NAME + FULL_MODEL_NAME + "agent_weights_" + '{:04d}'.format(int(e/CHECKPOINT_FREQUENCY)) +".hdf5")
          


#### Interactive Playground (runs automatically)

As the agent finally blitzed through the code, we are getting close to the deadline and what began as an awesome undertaking in abstraction might formally (and temporarily) become a pure comparision of how the transfer learned weights from notebook 1 affect training speed. However, there is still yet another problem, because with only 9 days left to finish, and each test taking a full 12 hours to see results the results come in and the model is still showing no progress. Luckily, one more day of debugging and preparation found a simple bug causing enourmous amounts of pre-buffer data to be labelled terminal. After this fix, the agent has begun to train comparably to the only other working example we found (linked above). This implementation is still slower, however, in that example the baselines wrapper provided by DeepMind is used, which contains significant speed and memory optimizations which unfortunately, only appear to offer frame stacking at 4 frames. Since this would impair our transfer learning basis this is currently the fastest implementation we can provide.

##### Launch

In [ ]:
# #### Run From Bottom
## This will take about 3 minutes in colab to gather starting data and output some feedback
## after this you can expect chippieBot limited updates about every 5 minutes for the next 7-12 hours.
myPlayground()

Starting Frames Acquired

Episode: 250/20000, Episode Score: 0, Avg Episode Score: 0.328, Survival Time: 22
Your agent has a running average Score per 100 episodes of: \{^,^}~0.38
Total Steps: 8665, Memory Size: 83664, Current Epsilon Value: 0.81, Leading Loss: 0.08927/ Trailing Loss: 0.0973

Episode: 500/20000, Episode Score: 0, Avg Episode Score: 0.31, Survival Time: 20
Your agent has a running average Score per 100 episodes of: \{^,^}~0.24
Total Steps: 17086, Memory Size: 92085, Current Epsilon Value: 0.66, Leading Loss: 0.08431/ Trailing Loss: 0.08808

Episode: 750/20000, Episode Score: 0, Avg Episode Score: 0.3053, Survival Time: 20
Your agent has a running average Score per 100 episodes of: \{^,^}~0.34
Total Steps: 25525, Memory Size: 100000, Current Epsilon Value: 0.54, Leading Loss: 0.07402/ Trailing Loss: 0.0796

Episode: 1000/20000, Episode Score: 1.0, Avg Episode Score: 0.295, Survival Time: 50
Your agent has a running average Score per 100 episodes of: \{^,^}~0.26
Total Ste

#### Output

When we are all done building this last box will zip our progress for download so we can continue our work later

In [ ]:
ZIP_OUTPUT = False #@param {type:"boolean"}

In [ ]:
%%capture
if ZIP_OUTPUT:
  zip = !zip -r ISAR_Model_Data_Output.zip ISAR_Model_Data